In [20]:
import os
import pandas as pd
from pandas import DataFrame

In [21]:
# Define the folder containing the CSV files
FOLDER_PATH = '../Crawl'
CSV_FILE = '../combined.csv'

In [22]:
 # List to hold dataframes
dfs = []

# Iterate over all files in the folder
for filename in os.listdir(FOLDER_PATH):
    if filename.endswith('.csv'):
        file_path = os.path.join(FOLDER_PATH, filename)
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all dataframes
combined_df = pd.concat(dfs, ignore_index=True)

# Drop unnecessary columns 
combined_df = combined_df.drop(columns=['page', 'name', 'link', 'image'])

In [23]:
def print_unique_values_with_counts(df: DataFrame, column: str, prefix: str):
    print(prefix)
    print(f"Column: {column}")
    print("Unique Values and Counts:")
    print(df[column].value_counts())
    print(f"Total Count: {df[column].count()}")
    print("-" * 100)

In [24]:
from mapping_brand import brand_mapping

def process_brand(data: DataFrame, column='brand') -> DataFrame:
    data[column] = data[column].replace(brand_mapping)
    data = data[~data[column].isin(['Prestige', 'kstation', 'gaming', 'Workstation', 'Samsung', 'iPad'])]
    
    return data

In [25]:
from mapping_cpu import cpu_mapping

def process_cpu(data: DataFrame, column='cpu') -> DataFrame:
    data[column] = data[column].str.replace("CPU ", "", regex=False)
    data[column] = data[column].replace(cpu_mapping)
    return data

In [26]:
from mapping_cpu_brand import cpu_brand_mapping

def process_cpu_brand(data: DataFrame, column='cpu_brand') -> DataFrame:
    data[column] = data[column].replace(cpu_brand_mapping)
    data = data[~data[column].isin(['CPU', 'Chip'])]
    
    return data

In [27]:
def process_ram_capacity(data: DataFrame, column='ram_capacity') -> DataFrame:
    data[column] = data[column].str.extract(r"(\d+)").astype(float)
    return data

In [28]:
from mapping_ram_brand import ram_brand_mapping

def process_ram_brand(data: DataFrame, column='ram_brand') -> DataFrame:
    data[column] = data[column].str.strip().replace(ram_brand_mapping)
    data = data[~data[column].isin(["3200", "5600", "4800", "5200", "7467", "6400", "7500", "1TB", "Soldered"])]
    return data

In [29]:
def process_hard_drive_capacity(data: DataFrame, column='hard_drive_capacity') -> DataFrame:
    data[column] = data[column].str.extract(r"(\d+)")[0].astype(float).apply(lambda x: f"{int(x)}TB" if x in [1, 2, 4] else ("1TB" if x == 1024 else (f"{int(x)}GB" if x in [128, 256, 512] else None)))
    return data

In [30]:
def process_card(data: DataFrame, column='card') -> DataFrame:
    return data

In [31]:
# Process brand
print_unique_values_with_counts(combined_df, column='brand', prefix='BEFORE')
combined_df = process_brand(combined_df)
print_unique_values_with_counts(combined_df, column='brand', prefix='AFTER')

BEFORE
Column: brand
Unique Values and Counts:
brand
Lenovo         320
Dell           140
Asus           130
HP              84
Acer            77
MSI             72
ASUS            40
LG              19
Gaming          17
Macbook         13
Microsoft       10
LENOVO           4
iPad             2
Samsung          2
Ideapad          2
Workstation      2
MacBook          2
Apple            2
ACER             2
gaming           1
ThinkPad         1
kstation         1
Prestige         1
Legion           1
Name: count, dtype: int64
Total Count: 945
----------------------------------------------------------------------------------------------------
AFTER
Column: brand
Unique Values and Counts:
brand
Lenovo       328
Asus         170
Dell         140
HP            84
Acer          79
MSI           72
LG            19
Macbook       17
Gaming        17
Microsoft     10
Name: count, dtype: int64
Total Count: 936
----------------------------------------------------------------------------------

In [39]:
# Process cpu
print_unique_values_with_counts(combined_df, column='cpu', prefix='BEFORE')
combined_df = process_cpu(combined_df)
print_unique_values_with_counts(combined_df, column='cpu', prefix='AFTER')

BEFORE
Column: cpu
Unique Values and Counts:
cpu
Intel Core Ultra 7 155H    60
Intel Core i5              37
Intel Core i7              36
Intel Core Ultra 7         29
Intel Core Ultra 5 125H    23
                           ..
Intel Core i7-9750H         1
Intel Core i7-10610u        1
Intel Core i9-12900H        1
Intel Core i9-13950HX       1
AMD Ryzen 5 4650U           1
Name: count, Length: 223, dtype: int64
Total Count: 927
----------------------------------------------------------------------------------------------------
['AMD Ryzen 9 AI Processor 365' 'AMD Ryzen 7 8845H'
 'Apple M4 Max (14-core & 16-core)' 'Apple M4 Pro (14-core)'
 'Apple M4 Max (14-core)' 'Apple M4 Pro (12-core & 14-core)'
 'AMD Ryzen 5 7520U' 'Intel Core i7-1355U' 'AMD Ryzen 7 8840HS'
 'Intel Core i7-13650HX' 'Intel Core i9-14900HX' 'AMD Ryzen 9 HX 370'
 'AMD Ryzen 5 7235HS' 'AMD Ryzen 5 8640HS' 'AMD Ryzen 7 7435HS'
 'Intel Core i5-12450HX' 'Intel Core i5-1340P' 'Intel Core Ultra 7 155H'
 'Intel Core Ultra 

In [33]:
# Process cpu_brand
print_unique_values_with_counts(combined_df, column='cpu_brand', prefix='BEFORE')
combined_df = process_cpu_brand(combined_df)
print_unique_values_with_counts(combined_df, column='cpu_brand', prefix='AFTER')

BEFORE
Column: cpu_brand
Unique Values and Counts:
cpu_brand
Intel         750
AMD           128
Ryzen          13
Apple          10
Core           10
Ultra           6
Snapdragon      5
Qualcomm        5
M3              2
CPU             1
M               1
i7              1
M2              1
i5-12500H       1
Chip            1
Intel;          1
Name: count, dtype: int64
Total Count: 936
----------------------------------------------------------------------------------------------------
AFTER
Column: cpu_brand
Unique Values and Counts:
cpu_brand
Intel       769
AMD         128
Apple        14
Ryzen        13
Qualcomm     10
Name: count, dtype: int64
Total Count: 934
----------------------------------------------------------------------------------------------------


In [34]:
# Process ram_capacity
print_unique_values_with_counts(combined_df, column='ram_capacity', prefix='BEFORE')
combined_df = process_ram_capacity(combined_df)
print_unique_values_with_counts(combined_df, column='ram_capacity', prefix='AFTER')

BEFORE
Column: ram_capacity
Unique Values and Counts:
ram_capacity
16GB     444
32GB     183
8GB      115
16 GB    104
8 GB      21
12GB      13
64GB      12
32 GB     12
24GB      11
36GB       4
12 GB      4
24 GB      4
           3
18GB       2
48GB       2
GB         1
Name: count, dtype: int64
Total Count: 935
----------------------------------------------------------------------------------------------------
AFTER
Column: ram_capacity
Unique Values and Counts:
ram_capacity
16.0    548
32.0    195
8.0     136
12.0     17
24.0     15
64.0     12
36.0      4
18.0      2
48.0      2
Name: count, dtype: int64
Total Count: 931
----------------------------------------------------------------------------------------------------


In [35]:
# Process ram_brand
print_unique_values_with_counts(combined_df, column='ram_brand', prefix='BEFORE')
combined_df = process_ram_brand(combined_df)
print_unique_values_with_counts(combined_df, column='ram_brand', prefix='AFTER')

BEFORE
Column: ram_brand
Unique Values and Counts:
ram_brand
DDR5            325
DDR4            228
LPDDR5          120
LPDDR5X          75
LPDDR5x          45
LPDDR4X          29
LPDDR3           11
LPDDR4            9
LPDDR4x           7
3200              7
5600              6
DDR5 4800MHz      5
DDR5-4800         4
4800              4
5200              4
DDR4-3200         3
LPDDR5x-7467      3
DDR5 4800         2
DDR5-5600         2
1TB               1
LPDDDR5           1
7467              1
Soldered          1
LPDDR5-6400       1
6400              1
7500              1
LPDDR5-5200       1
                  1
Name: count, dtype: int64
Total Count: 898
----------------------------------------------------------------------------------------------------
AFTER
Column: ram_brand
Unique Values and Counts:
ram_brand
DDR5       338
DDR4       231
LPDDR5X    123
LPDDR5     123
LPDDR4X     36
LPDDR3      11
LPDDR4       9
Name: count, dtype: int64
Total Count: 871
---------------------------

In [36]:
# Process hard_drive_capacity
print_unique_values_with_counts(combined_df, column='hard_drive_capacity', prefix='BEFORE')
combined_df = process_hard_drive_capacity(combined_df)
print_unique_values_with_counts(combined_df, column='hard_drive_capacity', prefix='AFTER')

BEFORE
Column: hard_drive_capacity
Unique Values and Counts:
hard_drive_capacity
512 GB     259
512GB      227
1TB        148
1          105
256 GB      48
256GB       24
512         17
B           13
2           11
2TB          9
SSD          4
1024         4
4TB          4
128 GB       2
128GB        2
(            2
5PA          1
6PA          1
1TB SSD      1
9PA          1
PCIe         1
256          1
Name: count, dtype: int64
Total Count: 885
----------------------------------------------------------------------------------------------------
AFTER
Column: hard_drive_capacity
Unique Values and Counts:
hard_drive_capacity
512GB    503
1TB      258
256GB     73
2TB       20
128GB      4
4TB        4
Name: count, dtype: int64
Total Count: 862
----------------------------------------------------------------------------------------------------


In [37]:
# Process card 
print_unique_values_with_counts(combined_df, column='card', prefix='BEFORE')
combined_df = process_card(combined_df)
print_unique_values_with_counts(combined_df, column='card', prefix='AFTER')

BEFORE
Column: card
Unique Values and Counts:
card
Intel Iris Xe Graphics                           133
Intel Arc Graphics                                88
Intel UHD Graphics                                68
Intel Graphics                                    41
AMD Radeon Graphics                               26
                                                ... 
NVIDIA RTX A4000 w/8GB                             1
NVIDIA RTX A2000 w/4GB Intel Iris Xe Graphics      1
NVidia Geforce RTX 4060 8GB                        1
NVidia Geforce RTX 4050                            1
Geforce RTX3080 16GB                               1
Name: count, Length: 245, dtype: int64
Total Count: 924
----------------------------------------------------------------------------------------------------
AFTER
Column: card
Unique Values and Counts:
card
Intel Iris Xe Graphics                           133
Intel Arc Graphics                                88
Intel UHD Graphics                                68


In [38]:
# Save the combined dataframe to a new CSV file
# combined_df.to_csv(CSV_FILE, index=False)